### This script sets up the model and performs all relevant model runs.

In [1]:
import sys, os

sys.path.append('C:\\Users\\ashreeta\\Documents\\Martin\\SHARED_REPOS')
print(os.getcwd())

import numpy as np
from importlib import reload

import grimsel.core.model_loop as model_loop
from grimsel.core.model_base import ModelBase as MB

import grimsel.auxiliary.aux_sql_func as aql
import grimsel.analysis.sql_analysis as sql_analysis

import grimsel.config as config
import grimsel.model_loop_modifier as model_loop_modifier

db = config.DATABASE

sc_out = 'out_replace_all'

C:\Users\ashreeta\Documents\Martin\SHARED_REPOS\grimsel


In [2]:
for fn in [fn for fn in os.listdir('.') if 'ephemeral' in fn or 'def_loop' in fn]:
    os.remove(fn)

In [3]:
connect_dict = dict(db=db,
                    password=config.PSQL_PASSWORD,
                    user=config.PSQL_USER,
                    port=config.PSQL_PORT,
                    host=config.PSQL_HOST)
sqlc = aql.sql_connector(**connect_dict)

# additional kwargs for the model
mkwargs = {
           'slct_encar': ['EL'],
           'slct_node': ['AT0', 'IT0', 'DE0', 'CH0', 'FR0'],
           'nhours': 1,
           'verbose_solver': False,
           'constraint_groups': MB.get_constraint_groups(excl=['chp', 'ror'])
           }

# additional kwargs for the i/o
iokwargs = {'resume_loop': 3581,
            'autocomplete_curtailment': True,
           }

nvr, nst = 30, 31
nsteps_default = [
                  ('swvr', nvr, np.arange),     # select vre share
                  ('swst', nst, np.arange),     # select storage capacity
                  ('swtc', 3, np.arange),       # select storage tech
                  ('swpt', 3, np.arange),       # select vre type
                  ('swyr', 5, np.arange),       # select meteo year
                  ('swco', 3, np.arange),       # select co2 emission price
                  ('swtr', 2, np.arange),       # cross-border transmission on/off
                  ('swrc', 26, np.arange)]      # select ramping cost

mlkwargs = {
            'sc_out': sc_out,
            'db': db,
            'nsteps': nsteps_default,
            'sql_connector': sqlc,
            }

sc_out = mlkwargs['sc_out']

ml = model_loop.ModelLoop(**mlkwargs, mkwargs=mkwargs, iokwargs=iokwargs)


swst_max = nst
dict_st = {nst: st for nst, st in enumerate(list(np.linspace(0, 0.3, swst_max)))}
swvr_max = nvr - 1
dict_vre = {nvr: vr for nvr , vr
            in enumerate(['default'] + list(np.linspace(0, 0.7, swvr_max)))}



self.slct_encar=['EL']
self.slct_pp_type=[]
self.slct_node=['AT0', 'IT0', 'DE0', 'CH0', 'FR0']
self.nhours=1
self.constraint_groups=['capacity', 'charging_level', 'chp_new', 'energy_aggregation', 'energy_constraint', 'hydro', 'monthly_total', 'objective', 'ramp_rate', 'supply', 'transmission', 'variables', 'yearly_cost']
Output schema:  out_replace_all ; resume loop= 3581
Reading input table def_plant 
Reading input table def_month 
Reading input table parameter_month 
Reading input table def_week 
Reading input table def_encar filtered by ca in ['EL']
Reading input table def_pp_type 
Reading input table def_node filtered by nd in ['AT0', 'IT0', 'DE0', 'CH0', 'FR0']
Reading input table def_plant filtered by nd_id in [0, 1, 2, 3, 4], pt_id in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Reading input table node_encar filtered by nd_id in [0, 1, 2, 3, 4], ca_id in [0]
Reading input table profprice filtered by nd_id in [0, 1, 2, 3, 4]
Reading input table profchp filtered by nd_i

C:\Users\ashreeta\Documents\Martin\SHARED_REPOS\grimsel\core\autocomplete.py:167: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  pd.concat([self._df, self.df_add[add_cols]]))


done. Added: AT_DMND, AT_DMND_FLEX, CH_DMND, CH_DMND_FLEX, DE_DMND, DE_DMND_FLEX, FR_DMND, FR_DMND_FLEX, IT_DMND, IT_DMND_FLEX
Autocompletion df_def_plant in AutoCompletePlantCons ... nothing added.
Autocompletion df_plant_encar in AutoCompletePpCaFlex ... done. Added: [77, 0], [79, 0], [81, 0], [83, 0], [85, 0]
############################################################
out_replace_all storage2
Generating time map from 2015-1-1 00:00 to 2015-12-31 23:59
Averaging table df_profdmnd to time resolution 1 hours.
Averaging table df_profprice to time resolution 1 hours.
Averaging table df_profinflow to time resolution 1 hours.
Averaging table df_profsupply to time resolution 1 hours.
Averaging table df_profchp to time resolution 1 hours.
---> storage2 out_replace_all def_pp_type pt
---> storage2 out_replace_all def_node nd
---> storage2 out_replace_all def_fuel fl
p1
get_ilst got superseeded by cols2tuplelist
get_ilst got superseeded by cols2tuplelist
get_ilst got superseeded by cols2tuple

Deleting from out_replace_all.var_yr_pwr_ramp_yr where run_id >= 3581
Deleting from out_replace_all.var_yr_vc_fl_pp_yr where run_id >= 3581
Deleting from out_replace_all.var_yr_vc_ramp_yr where run_id >= 3581
Deleting from out_replace_all.var_yr_vc_co2_pp_yr where run_id >= 3581
Deleting from out_replace_all.var_yr_vc_om_pp_yr where run_id >= 3581
Deleting from out_replace_all.var_yr_fc_om_pp_yr where run_id >= 3581
Deleting from out_replace_all.var_yr_fc_cp_pp_yr where run_id >= 3581
Deleting from out_replace_all.var_yr_fc_dc_pp_yr where run_id >= 3581
Deleting from out_replace_all.var_yr_vc_dmnd_flex_yr where run_id >= 3581
Deleting from out_replace_all.var_yr_cap_pwr_rem where run_id >= 3581
Deleting from out_replace_all.var_yr_cap_pwr_tot where run_id >= 3581
Deleting from out_replace_all.var_yr_cap_erg_tot where run_id >= 3581
Deleting from out_replace_all.var_yr_cap_pwr_new where run_id >= 3581
Deleting from out_replace_all.par_share_ws_set where run_id >= 3581
Deleting from out_

In [4]:
###############################################################################
# reduce output

ml.io.var_sy = [par for par in ml.io.var_sy if 'pwr' in par[0] or 'pwr_st_ch' in par[0]]
ml.io.par = [par for par in ml.io.par if not 'prof' in par[0]]
ml.io.var_tr = [var for var in ml.io.var_tr if 'erg' in var[0]]
ml.io.dual = []

###############################################################################

In [5]:
ml.io.var_sy

[('pwr', ('sy', 'pp_id', 'ca_id', 'bool_out')),
 ('pwr_st_ch', ('sy', 'pp_id', 'ca_id', 'bool_out'), 'pwr')]

### Some filtering of the ml.df_def_loop table so we don't end up doing 376650 model runs:

In [6]:

# figure 8/9: various nuclear power indicators france 
slct_vr = [0] + list(np.arange(0, ml.df_def_loop.swvr_id.max(), 4) + 1)
slct_st = list(np.arange(0, ml.df_def_loop.swvr_id.max() + 10, 10))
mask_base = (ml.df_def_loop.swvr_id.isin(slct_vr) &
               ml.df_def_loop.swst_id.isin(slct_st) &
               ml.df_def_loop.swtc_id.isin([0, 1]) &
               ml.df_def_loop.swpt_id.isin([0]) &
               ml.df_def_loop.swyr_id.isin([0]) &
               ml.df_def_loop.swco_id.isin([0]) &
               ml.df_def_loop.swtr_id.isin([0]) &
               ml.df_def_loop.swrc_id.isin([0]))

# figure 10: ramping costs
slct_vr = list(np.arange(13, ml.df_def_loop.swvr_id.max() + 1, 4))
slct_st = list(np.arange(0, ml.df_def_loop.swvr_id.max() + 10, 10))
mask_ramping = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0, 1]) &
                ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
                ml.df_def_loop.swco_id.isin([0]) &
                ml.df_def_loop.swtr_id.isin([0]) 
#               ml.df_def_loop.swrc_id.isin([0]) --> all
               )

# figure 11: qualitatively diverging storage impact
slct_vr = [{round(val * 10000) / 10000 if not val == 'default' else val: key
            for key, val in dict_vre.items()}[vr] for vr in [0.5, 0.7, 0.4]]
mask_years = (ml.df_def_loop.swvr_id.isin(slct_vr) &
#                ml.df_def_loop.swst_id.isin(slct_st) & --> all
                ml.df_def_loop.swtc_id.isin([0, 1]) &
                ml.df_def_loop.swpt_id.isin([0]) &
#                ml.df_def_loop.swyr_id.isin([0]) & --> all
                ml.df_def_loop.swco_id.isin([0]) &
                ml.df_def_loop.swtr_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )

# figure 12: consecutive replacement
slct_vr = [0] + list(np.arange(0, ml.df_def_loop.swvr_id.max(), 1) + 1)
slct_st = list(np.arange(0, ml.df_def_loop.swvr_id.max() + 5, 10))
mask_consec = (#ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0]) &
                ml.df_def_loop.swpt_id.isin([0, 1, 2]) &
                ml.df_def_loop.swyr_id.isin([0]) &
                ml.df_def_loop.swco_id.isin([0, 1]) &
                ml.df_def_loop.swtr_id.isin([0, 1]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )

# figure 13: emissions
slct_vr = [0] + list(np.arange(0, ml.df_def_loop.swvr_id.max(), 2) + 1)
slct_st = list(np.arange(0, ml.df_def_loop.swvr_id.max() + 100, 10))
mask_emissions = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0]) &
                ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
#                ml.df_def_loop.swco_id.isin([0]) & --> all
                ml.df_def_loop.swtr_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )

mask_total = mask_base | mask_ramping | mask_years | mask_consec | mask_emissions

print(mask_total.sum())


3446


In [7]:
# Extra for emissions plot

slct_vr = [0] + list(np.arange(0, ml.df_def_loop.swvr_id.max(), 2) + 1)
slct_st = list(np.arange(0, ml.df_def_loop.swst_id.max() + 100, 5))
mask_emissions = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0]) &
                ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
#                ml.df_def_loop.swco_id.isin([0]) & --> all
                ml.df_def_loop.swtr_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )


mask_emissions = mask_emissions & -mask_total

print(mask_emissions.sum())

135


In [19]:
# Extra for emissions plot LIO_STO

slct_vr = [0] + list(np.arange(0, ml.df_def_loop.swvr_id.max(), 2) + 1)
slct_st = [0, 20] # ??????????????  list(np.arange(0, ml.df_def_loop.swst_id.max() + 100, 5))
mask_emissions_LIO = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                      ml.df_def_loop.swst_id.isin(slct_st) &
                      ml.df_def_loop.swtc_id.isin([1]) & # <====================
                      ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
#                ml.df_def_loop.swco_id.isin([0]) & --> all
                ml.df_def_loop.swtr_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )


mask_emissions_LIO = mask_emissions_LIO & -mask_total& -mask_emissions

print(mask_emissions_LIO.sum())

78


In [20]:
import pandas as pd

ml.df_def_loop = pd.concat([ml.df_def_loop.loc[mask_total],
                            ml.df_def_loop.loc[mask_emissions],
                            ml.df_def_loop.loc[mask_emissions_LIO],
                           ])
ml.df_def_loop

,run_id,swvr_id,swst_id,swtc_id,swpt_id,swyr_id,swco_id,swtr_id,swrc_id,swvr,swst,swtc,swpt,swyr,swco,swtr,swrc,swvr_vl,swst_vl,swtc_vl,swpt_vl,swyr_vl,swco_vl,swtr_vl,swrc_vl
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# init ModelLoopModifier
mlm = model_loop_modifier.ModelLoopModifier(ml)

# starting row of loop
irow_0 = ml.io.resume_loop if ml.io.resume_loop else 0

irow = irow = 0

ml.m._limit_prof_to_cap()
#ml.perform_model_run(zero_run=True)
# %
for irow in list(range(irow_0, len(ml.df_def_loop))):
    run_id = irow

    print('select_run')
    ml.select_run(run_id)

    ###
    print('set_trm_cap_onoff')
    mlm.set_trm_cap_onoff()
    
    ####
    print('set_co2_price')
    mlm.set_co2_price()

    ####
    print('set_winsol_year')
    mlm.set_winsol_year()

    ####
    print('select_vre_pp_types')
    slct_pp, slct_pt = mlm.select_vre_pp_types()

    ####
    print('scale_vre')
    mlm.scale_vre(slct_pt, dict_vre)

    ####
    print('set_ramping_cost')
    mlm.set_ramping_cost()

    ####
    print('select_storage_tech')
    mlm.select_storage_tech()

    ####
    print('set_storage_cap')
    mlm.set_storage_cap(dict_st)

    ############### RUN MODEL ###############
    print('fill_peaker_plants')
    ml.m.fill_peaker_plants(demand_factor=2)

    print('_limit_prof_to_cap')
    ml.m._limit_prof_to_cap()

    print('perform_model_run')
    ml.perform_model_run()

    for fn in [fn for fn in os.listdir('.') if 'ephemeral' in fn or 'def_loop' in fn]:
        os.remove(fn)





************************************************************
ModelBase: Limiting chp profiles to cap_pwr_leg... 
(nd, ca, fl)=(0, 0, 13): 19.0 violations
Modifing model parameter cap_pwr_leg ... done.
************************************************************
select_run
set_trm_cap_onoff
set_co2_price
set_winsol_year
select_vre_pp_types
scale_vre
set_ramping_cost
select_storage_tech
set_storage_cap
fill_peaker_plants
_limit_prof_to_cap
************************************************************
ModelBase: Limiting chp profiles to cap_pwr_leg... ok, nothing changed.
************************************************************
perform_model_run
************************************************************
run_id = 3581 of 3658
swrc_vl = x1.0
swst_vl = 0.00%
swco_vl = 40EUR/tCO2
swtc_vl = LIO_STO
swpt_vl = WIN_ONS|WIN_OFF|SOL_PHO
swtr_vl = on
swvr_vl = 5.00%
swyr_vl = 2015
************************************************************
Using warmstartfile:  None
Using solutionfile:  C:\Use

C:\Users\ashreeta\Documents\Martin\SHARED_REPOS\grimsel\core\io.py:633: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _df['run_id'] = self.run_id


Writing to db:  out_replace_all.var_sy_pwr... done in 80.016 sec
Writing to db:  out_replace_all.var_sy_pwr... done in 8.524 sec
Writing to db:  out_replace_all.var_yr_erg_yr... done in 0.070 sec
Writing to db:  out_replace_all.var_yr_erg_fl_yr... done in 0.088 sec
Writing to db:  out_replace_all.var_yr_pwr_ramp_yr... done in 0.106 sec
Writing to db:  out_replace_all.var_yr_vc_fl_pp_yr... done in 0.069 sec
Writing to db:  out_replace_all.var_yr_vc_ramp_yr... done in 0.042 sec
Writing to db:  out_replace_all.var_yr_vc_co2_pp_yr... done in 0.077 sec
Writing to db:  out_replace_all.var_yr_vc_om_pp_yr... done in 0.078 sec
Writing to db:  out_replace_all.var_yr_fc_om_pp_yr... done in 0.043 sec
Component fc_cp_pp_yr does not exist... skipping.
Writing to db:  out_replace_all.var_yr_fc_dc_pp_yr... done in 0.065 sec
Writing to db:  out_replace_all.var_yr_vc_dmnd_flex_yr... done in 0.080 sec
Writing to db:  out_replace_all.var_yr_cap_pwr_rem... done in 0.061 sec
Writing to db:  out_replace_all.

Writing to db:  out_replace_all.var_sy_pwr... done in 4.272 sec
Writing to db:  out_replace_all.par_erg_inp... done in 0.010 sec
Writing to db:  out_replace_all.par_erg_chp... done in 0.008 sec
Writing to db:  out_replace_all.par_capchnge_max... done in 0.003 sec
Writing to db:  out_replace_all.par_price_co2... done in 0.007 sec
Writing to db:  out_replace_all.par_vc_fl... done in 0.104 sec
Writing to db:  out_replace_all.par_cap_avlb... done in 0.031 sec
Writing to db:  out_replace_all.var_tr_erg_trm_rv_yr... done in 0.007 sec
Writing to db:  out_replace_all.var_yr_erg_yr... done in 0.011 sec
Writing to db:  out_replace_all.var_tr_erg_trm_sd_yr... done in 0.006 sec
Writing to db:  out_replace_all.var_yr_erg_yr... done in 0.005 sec
select_run
set_trm_cap_onoff
set_co2_price
set_winsol_year
select_vre_pp_types
scale_vre
set_ramping_cost
select_storage_tech
set_storage_cap
fill_peaker_plants
_limit_prof_to_cap
************************************************************
ModelBase: Limiti

In [10]:
sqa = sql_analysis.SqlAnalysis(sc_out=sc_out, db=db)
sqa.build_tables_plant_run_quick()
sqa.build_table_plant_run_tot_balance(from_quick=True)




Join columns pt, pp_broad_cat from out_replace_all.def_pp_type to out_replace_all.analysis_plant_run_quick 7.408423662185669 seconds
Join columns swvr_vl, swtc_vl, swyr_vl, swpt_vl, swst_vl, swtr_vl, swrc_vl, swco_vl from out_replace_all.def_loop to out_replace_all.analysis_plant_run_quick 7.52543044090271 seconds
Join columns set_def_winsol, fl_id, pp from out_replace_all.def_plant to out_replace_all.analysis_plant_run_quick 8.552489042282104 seconds
Join columns fl from out_replace_all.def_fuel to out_replace_all.analysis_plant_run_quick 18.285045623779297 seconds
Join columns nd from out_replace_all.def_node to out_replace_all.analysis_plant_run_quick 18.282045602798462 seconds
Join columns swvr_vl, swtc_vl, swyr_vl, swpt_vl, swst_vl, swtr_vl, swrc_vl, swco_vl from out_replace_all.def_loop to out_replace_all.analysis_plant_run_tot_balance 10.943625926971436 seconds
Join columns nd from out_replace_all.def_node to out_replace_all.analysis_plant_run_tot_balance 9.483542442321777 secon

"\n                    DROP TABLE IF EXISTS\n                        out_replace_all.analysis_plant_run_tot_balance CASCADE;\n                    SELECT\n                        run_id, bool_out,\n                        (CASE WHEN bool_out = False THEN 1 ELSE -1 END ) * ABS(erg_yr_yr) AS erg_yr_yr_posneg,\n                        pp_broad_cat, pt, fl, nd, nd_id, ca_id, pp\n                    INTO out_replace_all.analysis_plant_run_tot_balance\n                    FROM out_replace_all.analysis_plant_run_quick;\n\n                    /* GRID LOSSES */\n                    INSERT INTO out_replace_all.analysis_plant_run_tot_balance\n                    SELECT\n                        prt.run_id, True AS bool_out,\n                        - SUM(pp_sign * erg_yr_yr * grid_losses) AS erg_yr_sy,\n                        'GRIDLOSSES' AS pp_broad_cat,\n                        'GRDLSS' AS pt, 'gridlosses' AS fl,\n                        dfnd.nd, grdlss.nd_id, prt.ca_id,\n                       

In [1]:
# EMISSIONS LIN

slct_run_id = aql.exec_sql('''
                           SELECT run_id FROM out_replace_all.def_loop
                           WHERE swvr_vl IN (SELECT GENERATE_SERIES(0, 100, 5) || '.00%')
                           AND swst_vl IN ('0.00%', '20.00%')
                           AND swrc_vl = 'x1.0'
                           AND swyr_vl = '2015'
                           AND swpt_vl = 'WIN_ONS|WIN_OFF|SOL_PHO'
                           AND swtr_vl = 'on'
                           ''', db=db)
slct_run_id = list(np.array(slct_run_id).flatten())

sqa = sql_analysis.SqlAnalysis(sc_out=sc_out, db=db, slct_run_id=slct_run_id)
sqa.analysis_emissions_lin()

NameError: name 'aql' is not defined

In [8]:
# FILTDIFF FOR FRENCH NUCLEAR
import grimsel.auxiliary.maps as maps

mps = maps.Maps(sc_out, db)

slct_nd_id = [mps.dict_nd_id[nd] for nd in ['FR0']]
slct_run_id = aql.exec                           _sql('''
                           SELECT run_id FROM out_replace_all.def_loop
                           WHERE swvr_vl IN (SELECT GENERATE_SERIES(0, 100, 10) || '.00%')
                           AND swst_vl IN ('0.00%', '20.00%')
                           AND swrc_vl = 'x1.0'
                           AND swyr_vl = '2015'
AND swpt_vl = 'WIN_ONS|WIN_OFF|SOL_PHO'
                           AND swtr_vl = 'on'
                           AND swco_vl = '40EUR/tCO2'
                           ''', db=db)
slct_run_id = list(np.array(slct_run_id).flatten())

reload(sql_analysis)
sqac_filt_chgdch = sql_analysis.SqlAnalysis(sc_out=sc_out, db=db,
                                            slct_pt=['NUC_ELC'],
                                            slct_run_id=slct_run_id,
                                            nd_id=slct_nd_id)
self = sqac_filt_chgdch
sqac_filt_chgdch.analysis_agg_filtdiff()


---> storage2 out_replace_all def_pp_type pt
---> storage2 out_replace_all def_node nd
---> storage2 out_replace_all def_fuel fl
p1


ProgrammingError: column tg.sy does not exist
LINE 1: ...son FROM out_replace_all.tm_soy_full AS src WHERE tg.sy = sr...
                                                             ^
HINT:  Perhaps you meant to reference the column "src.sy".
